# Encoding

In [3]:
import random
'''Hyperparameters configuration'''
#Convolutional layers
FSIZES = [1,2,3,4,5,6,7]
NFILTERS = [2,4,8,16,32]

#Pooling layers
PSIZES = [2,3,4,5]
PTYPE = ['max', 'avg']

#Fully connected layers
NEURONS = [4,8,16,32,64,128]
random.seed(5)

class Encoding:
    def __init__(self, minC, maxC, minF, maxF):

        self.n_conv = random.randint(minC, maxC)
        self.n_full = random.randint(minF, maxF)

        '''First level encoding'''
        self.first_level = []
        #Feature extraction part
        for i in range(self.n_conv):
            layer = {'type' : 'conv',
                     'nfilters' : random.choice(NFILTERS),
                     'fsize' : random.choice(FSIZES),
                     'pool' : random.choice(['max', 'avg', 'off']),
                     'psize' : random.choice(PSIZES)
                    }
            self.first_level.append(layer)

        #Fully connected part
        for i in range(self.n_full):
            layer = {'type' : 'fc',
                     'neurons' : random.choice(NEURONS)}
            self.first_level.append(layer)

        '''Second level encoding'''
        self.second_level = []
        prev = -1
        for i in range(self.n_conv):
            if prev < 1:
                prev += 1
            if prev >= 1:
                for _ in range(prev-1):
                    self.second_level.append(random.choice([0,1]))
                prev += 1

e = Encoding(2,8,1,4)

In [21]:
!pip install torch torchvision torchaudio torchtext
!pip install pytorch-forecasting
!pip install pandas
!pip install torchsummary
!pip install openpyxl
!pip install xlrd xlwt
#!pip uninstall torchvision
#!pip install torchvision


  Obtaining dependency information for pytorch-forecasting from https://files.pythonhosted.org/packages/43/80/79c0bb9f064c87cd282029f4db7bbdde333c91824c28b59739622770f542/pytorch_forecasting-0.10.1-py3-none-any.whl.metadata
  Using cached pytorch_forecasting-0.10.1-py3-none-any.whl.metadata (11 kB)
  Obtaining dependency information for optuna<3.0.0,>=2.3.0 from https://files.pythonhosted.org/packages/9f/6c/c6fab7d673d9d12144b8fe7155e15aeb06798b029ecfece4b193ffc859e6/optuna-2.10.1-py3-none-any.whl.metadata
  Using cached optuna-2.10.1-py3-none-any.whl.metadata (15 kB)
  Obtaining dependency information for pandas<2.0.0,>=1.3.0 from https://files.pythonhosted.org/packages/da/6d/1235da14daddaa6e47f74ba0c255358f0ce7a6ee05da8bf8eb49161aa6b5/pandas-1.5.3-cp311-cp311-win_amd64.whl.metadata
  Using cached pandas-1.5.3-cp311-cp311-win_amd64.whl.metadata (12 kB)
  Obtaining dependency information for pytorch-lightning<2.0.0,>=1.2.4 from https://files.pythonhosted.org/packages/77/ed/7d91e1958f0d

  error: subprocess-exited-with-error
  
  Preparing metadata (pyproject.toml) did not run successfully.
  exit code: 1
  
  [66 lines of output]
  Partial import of sklearn during the build process.
  setup.py:128: DeprecationWarning:
  
    `numpy.distutils` is deprecated since NumPy 1.23.0, as a result
    of the deprecation of `distutils` itself. It will be removed for
    Python >= 3.12. For older Python versions it will remain present.
    It is recommended to use `setuptools < 60.0` for those Python versions.
    For more details, see:
      https://numpy.org/devdocs/reference/distutils_status_migration.html
  
  
    from numpy.distutils.command.build_ext import build_ext  # noqa
  INFO: No module named 'numpy.distutils._msvccompiler' in numpy.distutils; trying from distutils
  Traceback (most recent call last):
    File "C:\Users\David Velazco\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
      main()
    File "C:\Us

## Librerias 

In [22]:
import torch
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from torchsummary import summary
from torch import optim
import torchvision
from torchvision import transforms, utils
import cv2
import random
import math
import scipy.io

## Códigos adjuntos

In [23]:
#from DataLoaderRGB import *
#from Training import *
from encoding import *
from decoding import *
from operators import *

## Sincrinizamos la GPU donde corramos el código

In [24]:
if torch.cuda.is_available():
  device = torch.device("cuda:0")
print(device)

cuda:0


## Preparamos el data set, y lo convertimos en tensor

In [25]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.labels = []  # Lista de etiquetas de clase
        labels_file = os.path.join(root_dir, "data.txt")
        with open(labels_file, 'r') as file:
            for line in file:
                parts = line.strip().split()
                label_degrees = float(parts[1])  # Etiqueta de clase en grados
                label_radians = np.deg2rad(label_degrees)  # Convertir a radianes
                self.labels.append(label_radians)
        self.image_files = [f for f in os.listdir(root_dir) if f.endswith(".jpg")]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_name)  # Abrir imagen en su formato original (RGB)

        # Redimensionar la imagen a 256x256
        #image = image.resize((256, 256), Image.BILINEAR)
        image = image.resize((256, 256), Image.Resampling.BILINEAR)

        image = np.array(image)
        label = self.labels[idx]

        sample = {"image": image, "label": label}

        if self.transform:
            sample = self.transform(sample)
        return sample

class ToTensor(object):
    def __call__(self, sample):
        image, label = sample["image"], sample["label"]
        image = image.transpose((2, 0, 1))
        return {"image": torch.from_numpy(image), "label": label}

def loading_data():
    
    root_dir = r'C:\Users\David Velazco\Desktop\MIA UV 2022-2024\Tesis\Pruebas\CNN-Steering Angle\dataset_self_driving_100_images'
    custom_dataset = CustomDataset(root_dir=root_dir, transform=transforms.Compose([ToTensor()]))
    train_size = int(0.7 * len(custom_dataset))
    test_size = len(custom_dataset) - train_size

    train_dataset, test_dataset = torch.utils.data.random_split(custom_dataset, [train_size, test_size])
    print('Data size training: ', len(train_dataset))
    print('Data size test: ', len(test_dataset))
    train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)

    return train_loader, test_loader

## Funciones de entrenamiento de CNN

In [26]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch.optim as optim
import matplotlib.pyplot as plt
import torch
import numpy as np
from sklearn.metrics import r2_score
def entrenamiento(device, cnn, max_epochs, loss_func, optimizer, train_dl):
    cnn.train()  # Configurando la red en modo de entrenamiento
    torch.set_default_dtype(torch.float32)  # Establecer el tipo de tensor por defecto

    for epoch in range(max_epochs):

        # Iterando sobre los batches
        for data in train_dl:
            # Leyendo datos del batch
            xb, yb = data['image'].to(device, dtype=torch.float32), data['label'].to(device, dtype=torch.float32)

            # Obteniendo salida de la CNN
            y_pred = cnn(xb)
            yb = yb.unsqueeze(1)

            # Obteniendo valor de la función de costo (MSE)
            loss = loss_func(y_pred, yb)

            # Obteniendo gradiente y hacer paso hacia atrás
            loss.backward()  # Paso hacia atrás (derivadas)
            optimizer.step()  # Ajustando pasos con optimizador
            optimizer.zero_grad(set_to_none=True)  # Reiniciando gradientes a 0

    return loss.item(), cnn  # Devuelve la pérdida final y la CNN entrenada

import torch
import numpy as np
from sklearn.metrics import r2_score

def validacion(device, cnn, test_dl):
    '''Inicializar perdida (Linea 1)'''
    cnn.eval()  # Configurando la red en modo evaluación

    '''Iterando sobre los batches (si el tamaño de batch es de 100, solo habrá un batch) (Linea 2)'''
    for i, data in enumerate(test_dl, 0):
        '''Leyendo datos del batch (Linea 3 y Linea 4)'''
        xb, yb = data['image'], data['label']  # Leyendo entrada y etiqueta
        xb = xb.type(torch.DoubleTensor).to(device, dtype=torch.float32)  # Moviendo entrada a GPU
        yb = yb.to(device, dtype=torch.float64)  # Moviendo etiqueta a GPU

        '''Obteniendo salida de la CNN (Linea 5)'''
        y_pred = cnn(xb).detach().cpu().numpy()
        y = yb.detach().cpu().numpy()

    # Calculando el coeficiente de determinación R^2
    r2 = r2_score(y, y_pred)

    MAPE = np.mean(np.abs((y) - (y_pred + 1)/(y)))
    SMAPE = np.mean(np.abs((y_pred) - (y))/(np.abs(y) + np.abs(y_pred)))
    return r2, MAPE, SMAPE



def training(device, model, n_epochs, loss_func, train_dl, test_dl, lr, w, max_params):

    #Number of parameters
    params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    # Move CNN to GPU
    model.to(device)

    #Optimizer
    opt = optim.Adam(model.parameters(), lr = lr)

    #Obtaining mse
    mse, cnn = entrenamiento(device, model, n_epochs, loss_func, opt, train_dl)

    R2, MAPE,SMAPE = validacion(device, cnn, test_dl)
    
    #Append results to multiprocessing list
    return MAPE,R2, params, mse,f

## Configuración de parametros del algoritmo genético

In [31]:
#Maximun and minimum numbers of layers to initialize networks

min_conv = 2
max_conv = 6
min_full = 1
max_full = 4

'''Genetic Algorithm Parameters'''
cr = 0.8 #Crossover rate
mr = 0.5 #Mutation rate
N = 10 #Population size
T = 5 #Number of generations
t_size = 5 #tournament size
w = 0.3 #penalization weight
max_params = 4e6
num_epochs =  10
lr = 1e-3

#Defining loss function
loss_func = torch.nn.MSELoss()

### Instancias de datos en entrenamiento para la CNN


In [32]:
train_dl, test_dl = loading_data()

Data size training:  70
Data size test:  30


## Verificamos la dimensionalidad de los tensores

In [33]:
""" Dimensiones del tensor test"""
for i_batch,sample_batched in enumerate(test_dl):
  print(sample_batched['image'].shape)
  print(sample_batched['label'])
  break

torch.Size([10, 3, 256, 256])
tensor([ 0.0370, -0.0141, -0.0124,  0.0387,  0.0000,  0.0000,  0.0298,  0.1232,
         0.0000,  0.0264], dtype=torch.float64)


## Creación de la población en el algoritmo genético

In [34]:
import timeit
import pickle
import pandas as pd

'''Initialize population'''
print('Initialize population')
start = timeit.default_timer()
pop = []
mape = []
R = []
bestFitness = []
bestParams = []
bestMSE = []

while len(pop) < N:

    #Creating genomes (genetic encoding)
    e1 = Encoding(min_conv,max_conv,min_full,max_full)

    #Decoding the networks
    network1 = decoding(e1)

    #Creating the CNNs
    cnn1 = CNN(e1, network1[0], network1[1], network1[2])
    MAPE,R2, params, mse,f=  training(device, cnn1, num_epochs, loss_func, train_dl, test_dl, lr, w, max_params)
    print("Paramameters:{}, MAPE:{}".format(params, MAPE))
    pop.append([e1,MAPE, params, mse])
print('Finish')


Initialize population
Paramameters:30500801, MAPE:0.05393765475581225
Paramameters:430741, MAPE:0.2585938966116701
Paramameters:131693, MAPE:0.05668148641615921
Paramameters:1969753, MAPE:0.1775802787587987
Paramameters:65343, MAPE:0.04032496979453134
Paramameters:7009577, MAPE:0.25868186995944725
Paramameters:33105, MAPE:0.028364491778065104
Paramameters:3723529, MAPE:0.06367921290730327
Paramameters:330777, MAPE:0.0942950341162987
Paramameters:106289, MAPE:0.015300288238282023
Finish


## Algoritomo genético

In [35]:
'''Genetic Algorithm'''

for t in range(T):
    print('Generation: ', t)

    #Parents Selection
    parents = []
    while len(parents) < int(N/2):
        #Tournament Selection
        tournament = random.sample(pop, t_size)
        p1 = selection(tournament, 'min')
        tournament = random.sample(pop, t_size)
        p2 = selection(tournament, 'min')

        while p1 == p2:
            tournament = random.sample(pop, t_size)
            p2 = selection(tournament, 'min')

        parents.append(p1)
        parents.append(p2)

    #Reproduction
    offspring = []
    while len(offspring) < int(N/2):
        par = random.sample(parents, 2)
        #Crossover + Mutation
        if cr >= random.uniform(0,1): #Crossover
            p1 = par[0][0]
            p2 = par[1][0]
            c1, c2 = crossover(p1, p2)

            #Mutation
            if mr >= random.uniform(0,1):
                mutation(c1)

            if mr >= random.uniform(0,1):
                mutation(c2)


            #Decoding the network
            network1 = decoding(c1)
            network2 = decoding(c2)

            #Creating the CNN
            cnn1 = CNN(c1, network1[0], network1[1], network1[2])
            cnn2 = CNN(c2, network2[0], network2[1], network2[2])

            #Evaluate individuals
            MAPE,R2, params, mse,f=  training(device, cnn1, num_epochs, loss_func, train_dl, test_dl, lr, w, max_params)
            offspring.append([e1,MAPE, params, mse])

            MAPE,R2, params, mse,f=  training(device, cnn1, num_epochs, loss_func, train_dl, test_dl, lr, w, max_params)
            offspring.append([e1,MAPE, params, mse])

    #Replacement with elitism
    pop = pop + offspring
    pop.sort(key = lambda x: x[1])
    pop = pop[:N]

    leader = min(pop, key = lambda x: x[1])
    bestMSE.append(leader[1])
    mape.append(leader[2])
    bestParams.append(leader[3])

    print('Best MSE: ', leader[3])
    print('Best MAPE: ', leader[1])
    print('Best No. of Params: ', leader[2])
    print('No. of Conv. Layers: ', leader[0].n_conv)
    print('No. of FC Layers: ', leader[0].n_full)
    print('--------------------------------------------')

Generation:  0
Best MSE:  0.00013553407916333526
Best MAPE:  0.015300288238282023
Best No. of Params:  106289
No. of Conv. Layers:  4
No. of FC Layers:  3
--------------------------------------------
Generation:  1
Best MSE:  0.00013553407916333526
Best MAPE:  0.015300288238282023
Best No. of Params:  106289
No. of Conv. Layers:  4
No. of FC Layers:  3
--------------------------------------------
Generation:  2
Best MSE:  0.00013553407916333526
Best MAPE:  0.015300288238282023
Best No. of Params:  106289
No. of Conv. Layers:  4
No. of FC Layers:  3
--------------------------------------------
Generation:  3
Best MSE:  0.00013553407916333526
Best MAPE:  0.015300288238282023
Best No. of Params:  106289
No. of Conv. Layers:  4
No. of FC Layers:  3
--------------------------------------------
Generation:  4
Best MSE:  0.00013553407916333526
Best MAPE:  0.015300288238282023
Best No. of Params:  106289
No. of Conv. Layers:  4
No. of FC Layers:  3
--------------------------------------------


## Guardamos los datos 

In [36]:
import numpy as np
import pandas as pd

results = pd.DataFrame(list(zip(bestMSE,mape, bestParams)), columns = ['MSE', 'MAPE', 'No. Params'])
final_networks = []
final_connections = []
objects = []
for member in pop:
    p = member[0]
    objects.append(p)
    n_conv = p.n_conv
    n_full = p.n_full
    description = 'The network has ' + str(n_conv) + ' convolutional layers ' + 'with: '
    for i in range(n_conv):
        nfilters = str(p.first_level[i]['nfilters'])
        fsize = str(p.first_level[i]['fsize'])
        pool = str(p.first_level[i]['pool'])
        psize = str(p.first_level[i]['psize'])
        layer = '(' + nfilters + ', ' + fsize + ', ' + pool + ', ' + psize + ') '
        description += layer
    description += 'and '
    description += str(n_full)
    description += ' '
    description += 'fully-connected layers with: '
    for i in range(n_conv, n_conv+n_full):
        neurons = str(p.first_level[i]['neurons'])
        layer = '(' + neurons + ')'
        description += layer
    description += ' neurons'
    final_networks.append(description)
    
    connections = ''
    for bit in p.second_level:
        if bit == 1:
            connections += 'one - '
        if bit == 0:
            connections += 'zero - '
    final_connections.append(connections)

     
final_population = pd.DataFrame(list(zip(final_networks, final_connections)), columns = ['Network Architecture', 'Connections'])
'''Saving Results as CSV'''
final_population.to_csv('arquithecture.csv', index = False)
results.to_csv('results.csv', index = False)
stop = timeit.default_timer()
execution_time = (stop-start)/3600
print("Execution time: ", execution_time)

Execution time:  0.03194617652777298
